In [1]:
import cv2
import base64
import torch
import torchvision
import pandas as pd
import numpy as np

from services.service import get_label
from services.augmentations import  get_infer_augmentation
from services.model import FasterRCNN, InferImages


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
%matplotlib inline

In [2]:
n_classes = 5
det_model = FasterRCNN(n_classes)
det_model = det_model.load_from_checkpoint(checkpoint_path='../lightning_logs/detection/3class_v1/epoch=88-step=4984.ckpt', n_classes=n_classes)
det_model.eval()
det_model = det_model.to('cpu')

In [3]:
def readb64(uri):
   encoded_data = uri.split(',')[1]
   nparr = np.fromstring(base64.b64decode(encoded_data), np.uint8)
   img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
   return img

In [58]:
def process_infer(image):
    batch_size = 1
    outs = det_model(image)
    bboxes = outs[batch_size - 1]['boxes']
    labels = outs[batch_size - 1]['labels']
    scores = outs[batch_size - 1]['scores']
    return bboxes, scores, labels

In [ ]:
PATH = '/home/tim/work/tashkent/traffic/cutting_video/'

In [59]:
infer = InferImages(2, get_infer_augmentation)

In [60]:
infer.append([PATH + 'out-223.jpg'])
image = infer()

In [7]:
import sort
mot_tracker = sort.Sort(min_hits=2, iou_threshold=0) 

In [8]:
list_of_track = []
for i in range(12, 49):
    print(i)
    infer.append([PATH + f'out-2{i}.jpg'])
    image = infer()
    bboxes, scores, labels = process_infer(image)
    if len(scores) >= 1:
        bb = bboxes.detach().to(int).numpy()
        scores = scores.detach().numpy()
        det = np.append(bb, scores)
        uns_det = np.expand_dims(det, axis=0)
        print(uns_det)
        ret = mot_tracker.update(uns_det)
        list_of_track.append(ret)
    else:
        ret = mot_tracker.update()

    print(ret)
    


12
[]
13
[]
14
[[354.         220.         374.         242.           0.94769889]]
[]
15
[[361.         214.         382.         239.           0.99252582]]
[]
16
[[369.         207.         393.         234.           0.99913687]]
[[369.00542952 207.07811791 392.75803495 234.09911181   1.        ]]
17
[[377.         199.         404.         229.           0.99930155]]
[[377.16919327 199.19793854 403.65023088 229.09031365   1.        ]]
18
[[386.         189.         417.         221.           0.99984121]]
[[386.29849406 189.41831656 415.90996671 221.93535205   1.        ]]
19
[[396.         174.         432.         211.           0.99989831]]
[[396.29770419 175.92957891 430.08717458 212.32432363   1.        ]]
20
[[409.         153.         449.         197.           0.99992406]]
[[408.34573928 157.4178263  446.57480266 198.83417039   1.        ]]
21
[[426.        125.        474.        177.          0.9999553]]
[[424.45501697 132.22394419 468.72707796 180.18662552   1.        